<a href="https://colab.research.google.com/github/weso500/IOT-Anomaly-Detection/blob/main/Single_Image_VAE_B_FLUX_GenLatent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtained intially from https://keras.io/examples/generative/vae/ and https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/generative/ipynb/vae.ipynb

In [1]:
pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

Use this notebook as example for obtaining images: https://colab.research.google.com/drive/1EOFDisibXb9a4A3lZ_2z4Jbhlnx0OVy6 The below takes care of obtaining images

In [4]:
latent_dim = 3

encoder_inputs = keras.Input(shape=(256, 256, 3))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 32)   │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 64, 64, 64)     │         18,496 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 262144)         │              0 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │     33,554,560 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_mean (Dense)            │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_log_var (Dense)         │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sampling (Sampling)       │ (None, 3)              │              0 │ z_mean[0][0],          │
│                           │                        │                │ z_log_var[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 33,574,726 (128.08 MB)

 Trainable params: 33,574,726 (128.08 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(64 * 64 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((64, 64, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 3)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 262144)              │       1,048,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 256, 256, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 256, 256, 3)         │             867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,104,835 (4.21 MB)

 Trainable params: 1,104,835 (4.21 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from tensorflow.keras.layers import Layer
class KLLossLayer(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

        return kl_loss

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.kl_loss_layer = KLLossLayer()

    def call(self, inputs):  # Add the call method
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        return reconstructed

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = ops.mean(
                ops.sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = self.kl_loss_layer([z_mean, z_log_var])
        return reconstructed, kl_loss, z_mean, z_log_var

    def predict(self, x):
        reconstructed, kl_loss, z_mean, z_log_var = super().predict(x)
        return reconstructed, kl_loss, z_mean, z_log_var


In [7]:
!unzip /content/Gold_Folder_Final.zip

Streaming output truncated to the last 5000 lines.
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/108Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/109Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/10Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/110Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/111Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/112Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/113Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/114Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/115Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGB

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_B-FLUX',
  image_size=(256, 256),
  batch_size=32,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds = train_ds.map(lambda x: normalization_layer(x))

Found 500 files.


In [9]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_test_ds = test_ds.map(lambda x: normalization_layer(x))

Found 190 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT

In [10]:
test_anomaly_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Anomaly/Spectrograms_Anomaly_B-FLUX',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_anomaly_test_ds = test_anomaly_ds.map(lambda x: normalization_layer(x))

Found 182 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT-I/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_B+_IGBT

In [11]:
#(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
#mnist_digits = np.concatenate([x_train, x_test], axis=0)
#mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(normalized_ds, epochs=15, batch_size=32)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 27s 647ms/step - kl_loss: 103.1013 - loss: 44498.0898 - reconstruction_loss: 44394.9883
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - kl_loss: 419.1761 - loss: 27140.7559 - reconstruction_loss: 26721.5801
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - kl_loss: 268.9603 - loss: 23622.9395 - reconstruction_loss: 23353.9785
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - kl_loss: 152.0067 - loss: 22819.5195 - reconstruction_loss: 22667.5137
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - kl_loss: 120.4792 - loss: 22087.7891 - reconstruction_loss: 21967.3105
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - kl_loss: 113.2216 - loss: 20644.7617 - reconstruction_loss: 20531.5391
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - kl_loss: 113.7705 - loss: 19155.1289 - reconstruction_loss: 19041.3594
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - kl_loss: 111.6727 - loss: 18132.3203 - reconstruction_loss: 18020.6484
Epoch 

In [12]:
train_ds_test_ = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_B-FLUX',
  image_size=(256, 256),
  batch_size=1,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds_test1 = train_ds_test_.map(lambda x: normalization_layer(x))

Found 500 files.


In [13]:
pip install sewar

  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11419 sha256=e83e41c93793d1b01dfce74c18ae0481cb6e8f591a81be8c851a951c32719e5b
  Stored in directory: /root/.cache/pip/wheels/3f/af/02/9c6556ba287b62a945d737def09b8b8c35c9e1d82b9dfae84c
Successfully built sewar


In [14]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_ds_test1.take(500):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list



1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
0.0014053624218805336
62.18153
[[ 5.4081697  5.645146  -7.4784265]]
[[-2.9696774 -3.285046  -3.9576423]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
0.0014644783513078092
60.50019
[[ 5.3345156  5.5800004 -7.357185 ]]
[[-2.939806  -3.2771447 -3.9519687]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
0.0013581480871650677
60.42903
[[ 5.376133   5.5282826 -7.3585973]]
[[-2.936668  -3.2811942 -3.9159927]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
0.0013877869084667849
60.850792
[[ 5.371875   5.5619173 -7.393236 ]]
[[-2.935549  -3.2835796 -3.920115 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
0.001585885802229672
57.66958
[[ 5.282246   5.3818197 -7.166757 ]]
[[-2.8894815 -3.2511141 -3.8545313]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
0.0018910870566673451
56.387577
[[ 5.5280924  4.9272723 -7.1428657]]
[[-2.7541502 -3.4610262 -3.5786545]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
0.0015225284768230856
59.691116
[[ 5.310519   5.5438066 -7.2923436]]
[[-2.943052  -3.2677715 -3.947638 ]

In [15]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df = pd.DataFrame(dict)

# saving the dataframe
df.to_csv('B+_FLUX_train.csv')

In [16]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.021095052306349203
34.94394
[[ 4.0180287  4.1660376 -5.468695 ]]
[[-1.888477  -3.7677915 -3.6234608]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.020348161315425845
34.172215
[[ 3.96443   4.116444 -5.405299]]
[[-1.839108  -3.8033245 -3.614744 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.020240764125877584
33.331715
[[ 3.9178863  4.052795  -5.3363976]]
[[-1.7879562 -3.8114696 -3.5950072]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.020354612405885413
33.118702
[[ 3.9020953  4.0403275 -5.3184385]]
[[-1.7771196 -3.8156028 -3.5895414]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.02127277013232896
32.774292
[[ 3.8809047  4.0143743 -5.2894444]]
[[-1.7509227 -3.833169  -3.5867026]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.020105611701304062
32.824398
[[ 3.8814244  4.0216413 -5.2936215]]
[[-1.7606814 -3.8222435 -3.5827827]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.020912409592496104
32.903996
[[ 3.8877773  4.02591   -5.3003445]]
[[-1.7640246 -3.821409  -3.5851576]]
1/1 

In [17]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df2 = pd.DataFrame(dict)

# saving the dataframe
df2.to_csv('B+FLUX_testnorm.csv')

In [18]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_anomaly_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.05306596879882469
22.910767
[[ 3.3881836  3.0154161 -4.3811774]]
[[-0.83193797 -4.249469   -3.4930027 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.006999687576147835
37.663395
[[ 4.2322135  4.3218117 -5.6881676]]
[[-2.0924075 -3.5086029 -3.6001964]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.019513782079547685
26.730812
[[ 3.4762328  3.5902035 -4.755133 ]]
[[-1.4484464 -3.7740188 -3.3615377]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.00166166286908602
54.00644
[[ 5.439218   4.744906  -7.0027995]]
[[-2.5776205 -3.6165059 -3.548742 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.020643688199324144
23.34363
[[ 3.2613516  3.2652085 -4.425824 ]]
[[-1.1349642 -3.9584496 -3.3316643]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.02498784214020074
24.402035
[[ 3.3024328  3.414197  -4.5318327]]
[[-1.3116355 -3.7814338 -3.2809274]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.0016104827297267674
58.363853
[[ 5.625304  4.984056 -7.297174]]
[[-2.767562  -3.5261114 -3.580321 ]]
1/1

In [19]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df3 = pd.DataFrame(dict)

# saving the dataframe
df3.to_csv('B+FLUX_testanom.csv')

In [20]:
from google.colab import files
files.download('/content/B+FLUX_testanom.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
files.download('/content/B+FLUX_testnorm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
files.download('/content/B+_FLUX_train.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>